In [1]:
from io import BytesIO
from pathlib import Path
from typing import Any

import instructor
import openai
import requests
from dotenv import load_dotenv
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from PIL import Image
from sympy import preview
from .vu_dash import remove_asy

from dreamai.ai import ModelName, assistant_message, system_message, user_message
from dreamai.g_apis.auth import authenticate
from dreamai.g_apis.slides import add_slide, create_presentation, create_service

load_dotenv()

ask_oai = instructor.from_openai(openai.OpenAI())

SLIDES_API_VERSION = "v1"
DRIVE_API_VERSION = "v3"

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
TOKEN_FILE = "dai_slides_drive_tokens.json"
CLIENT_SECRET_FILE = "dai_slides_drive_creds.json"

In [12]:
def upload_images_to_drive(
    drive_service: Any, file_paths: str | Path | list[str | Path], folder_id: str
) -> list[str]:
    urls = []
    if not isinstance(file_paths, list):
        file_paths = [file_paths]
    for file_path in file_paths:
        file_path = str(file_path)
        base_name = Path(file_path).stem
        extension = Path(file_path).suffix
        mime_type = "image/jpeg"

        # Check if a file with the same name already exists in the folder
        response = (
            drive_service.files()
            .list(
                q=f"name='{base_name}{extension}' and '{folder_id}' in parents",
                spaces="drive",
                fields="files(id, name)",
            )
            .execute()
        )
        files = response.get("files", [])

        # If a file with the same name exists, add a suffix to the name
        if files:
            file_metadata = {
                "name": f"{base_name}_{len(files)}{extension}",
                "mimeType": mime_type,
                "parents": [folder_id],
            }
        else:
            file_metadata = {
                "name": f"{base_name}{extension}",
                "mimeType": mime_type,
                "parents": [folder_id],
            }

        media = MediaFileUpload(file_path, mimetype=mime_type, resumable=True)
        file = (
            drive_service.files()
            .create(body=file_metadata, media_body=media, fields="id")
            .execute()
        )

        permission = {
            "type": "anyone",
            "role": "reader",
        }
        drive_service.permissions().create(fileId=file["id"], body=permission).execute()

        file_url = (
            drive_service.files()
            .get(fileId=file["id"], fields="webContentLink")
            .execute()["webContentLink"]
        )
        urls.append(file_url)
    return urls


def create_drive_folder(drive_service: Any, folder_name: str) -> str:
    file_metadata = {
        "name": folder_name,
        "mimeType": "application/vnd.google-apps.folder",
    }
    file = drive_service.files().create(body=file_metadata, fields="id").execute()
    return file.get("id")


def create_slide(slides_service: Any, presentation_id: str, page_id: str):
    # Check if the page_id exists, if not, create a new slide
    presentation = (
        slides_service.presentations().get(presentationId=presentation_id).execute()
    )
    slide_ids = [slide["objectId"] for slide in presentation.get("slides", [])]
    if page_id not in slide_ids:
        # Create a new slide
        req = {"createSlide": {"objectId": page_id, "insertionIndex": len(slide_ids)}}
        body = {"requests": [req]}
        slides_service.presentations().batchUpdate(
            presentationId=presentation_id, body=body
        ).execute()


def add_title(
    slides_service: Any,
    presentation_id: str,
    page_id: str,
    title_id: str,
    title: str,
    title_height: int,
    slide_width: int,
    current_y: int,
):
    # Add title text box
    req = {
        "createShape": {
            "objectId": title_id,
            "shapeType": "TEXT_BOX",
            "elementProperties": {
                "pageObjectId": page_id,
                "size": {
                    "height": {"magnitude": title_height, "unit": "EMU"},
                    "width": {"magnitude": slide_width, "unit": "EMU"},
                },
                "transform": {
                    "scaleX": 1,
                    "scaleY": 1,
                    "translateX": 0,
                    "translateY": current_y,
                    "unit": "EMU",
                },
            },
        }
    }
    body = {"requests": [req]}
    slides_service.presentations().batchUpdate(
        presentationId=presentation_id, body=body
    ).execute()

    # Insert text into the text box
    req = {
        "insertText": {
            "objectId": title_id,
            "insertionIndex": 0,
            "text": title,
        }
    }
    body = {"requests": [req]}
    slides_service.presentations().batchUpdate(
        presentationId=presentation_id, body=body
    ).execute()


def add_image(
    slides_service: Any,
    presentation_id: str,
    page_id: str,
    image_id: str,
    image_url: str,
    slide_width: int,
    slide_image_width: int,
    slide_image_height: int,
    current_y: int,
):
    # Add image
    req = {
        "createImage": {
            "objectId": image_id,
            "url": image_url,
            "elementProperties": {
                "pageObjectId": page_id,
                "size": {
                    "height": {"magnitude": slide_image_height, "unit": "EMU"},
                    "width": {"magnitude": slide_image_width, "unit": "EMU"},
                },
                "transform": {
                    "scaleX": 1,
                    "scaleY": 1,
                    "translateX": (slide_width - slide_image_width)
                    / 2,  # Center the image horizontally
                    "translateY": current_y,
                    "unit": "EMU",
                },
            },
        }
    }
    body = {"requests": [req]}
    slides_service.presentations().batchUpdate(
        presentationId=presentation_id, body=body
    ).execute()


def add_images(
    slides_service: Any,
    presentation_id: str,
    page_ids: str | list[str],
    image_urls: str | list[str],
    image_titles: str | list[str],
):
    if isinstance(image_urls, str):
        image_urls = [image_urls]
    if isinstance(image_titles, str):
        image_titles = [image_titles]
    if isinstance(page_ids, str):
        page_ids = [page_ids]

    # Ensure the number of page_ids matches the number of image_urls and image_titles
    if len(page_ids) < len(image_urls):
        base_page_id = page_ids[0]
        for i in range(len(page_ids), len(image_urls)):
            page_ids.append(f"{base_page_id}_{i+1}")

    # Retrieve the presentation details to get the slide dimensions
    presentation = (
        slides_service.presentations().get(presentationId=presentation_id).execute()
    )
    page_size = presentation["pageSize"]
    slide_height = page_size["height"]["magnitude"]  # Height of the slide in EMUs
    slide_width = page_size["width"]["magnitude"]  # Width of the slide in EMUs

    # Calculate the vertical spacing and heights for the titles and images
    total_available_height = slide_height * 0.9  # Leave some margin at top and bottom
    spacing = total_available_height * 0.1
    title_height = spacing  # Height of the title text box

    for i, (page_id, image_url, title) in enumerate(
        zip(page_ids, image_urls, image_titles)
    ):
        current_y = spacing
        image_id = f"MyImage_{i+1}"
        text_id = f"MyText_{i+1}"

        create_slide(slides_service, presentation_id, page_id)

        add_title(
            slides_service=slides_service,
            presentation_id=presentation_id,
            page_id=page_id,
            title_id=text_id,
            title=title,
            title_height=title_height,
            slide_width=slide_width,
            current_y=current_y,
        )
        current_y += title_height + spacing  # Move down for the image

        # Get the dimensions of the image
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        image_width, image_height = image.size

        # Calculate the aspect ratio of the image
        aspect_ratio = image_width / image_height

        # Calculate the width and height of the image on the slide
        slide_image_width = slide_width - 2 * spacing
        slide_image_height = slide_width / aspect_ratio
        available_height = total_available_height - title_height - spacing
        if slide_image_height > available_height:
            slide_image_height = available_height
            slide_image_width = slide_height * aspect_ratio

        add_image(
            slides_service=slides_service,
            presentation_id=presentation_id,
            page_id=page_id,
            image_id=image_id,
            image_url=image_url,
            slide_width=slide_width,
            slide_image_width=slide_image_width,
            slide_image_height=slide_image_height,
            current_y=current_y,
        )

    print("Images and titles added successfully")

In [4]:
# def create_slide(slides_service: Any, presentation_id: str, page_id: str):
#     # Check if the page_id exists, if not, create a new slide with TITLE_AND_BODY layout
#     presentation = (
#         slides_service.presentations().get(presentationId=presentation_id).execute()
#     )
#     slide_ids = [slide["objectId"] for slide in presentation.get("slides", [])]
#     if page_id not in slide_ids:
#         # Create a new slide with TITLE_AND_BODY layout
#         req = {
#             "createSlide": {
#                 "objectId": page_id,
#                 "insertionIndex": len(slide_ids),
#                 "slideLayoutReference": {"predefinedLayout": "TITLE_AND_BODY"},
#             }
#         }
#         body = {"requests": [req]}
#         slides_service.presentations().batchUpdate(
#             presentationId=presentation_id, body=body
#         ).execute()


# def add_title(
#     slides_service: Any,
#     presentation_id: str,
#     page_id: str,
#     title: str,
# ):
#     # Insert text into the title placeholder
#     req = {
#         "insertText": {
#             "objectId": f"{page_id}_title",
#             "insertionIndex": 0,
#             "text": title,
#         }
#     }
#     body = {"requests": [req]}
#     slides_service.presentations().batchUpdate(
#         presentationId=presentation_id, body=body
#     ).execute()


# def add_image(
#     slides_service: Any,
#     presentation_id: str,
#     page_id: str,
#     image_url: str,
# ):
#     # Add image to the body placeholder
#     req = {
#         "createImage": {
#             "objectId": f"{page_id}_body",
#             "url": image_url,
#             "elementProperties": {
#                 "pageObjectId": page_id,
#             },
#         }
#     }
#     body = {"requests": [req]}
#     slides_service.presentations().batchUpdate(
#         presentationId=presentation_id, body=body
#     ).execute()


# def add_images(
#     slides_service: Any,
#     presentation_id: str,
#     page_ids: str | list[str],
#     image_urls: str | list[str],
#     image_titles: str | list[str],
# ):
#     if isinstance(image_urls, str):
#         image_urls = [image_urls]
#     if isinstance(image_titles, str):
#         image_titles = [image_titles]
#     if isinstance(page_ids, str):
#         page_ids = [page_ids]

#     # Ensure the number of page_ids matches the number of image_urls and image_titles
#     if len(page_ids) < len(image_urls):
#         base_page_id = page_ids[0]
#         for i in range(len(page_ids), len(image_urls)):
#             page_ids.append(f"{base_page_id}_{i+1}")

#     for i, (page_id, image_url, title) in enumerate(
#         zip(page_ids, image_urls, image_titles)
#     ):
#         create_slide(slides_service, presentation_id, page_id)
#         add_title(slides_service, presentation_id, page_id, title)
#         add_image(slides_service, presentation_id, page_id, image_url)

#     print("Images and titles added successfully")

In [5]:
problem_1 = remove_asy(
    'Let $\\theta$ be the angle between the line\n\\[\\frac{x + 1}{2} = \\frac{y}{3} = \\frac{z - 3}{6}\\]and the plane $-10x - 2y + 11z = 3.$  Find $\\sin \\theta.$\n\n[asy]\nimport three;\n\nsize(150);\ncurrentprojection = perspective(6,3,2);\n\ntriple I = (1,0,0), J = (0,1,0), K = (0,0,1), O = (0,0,0);\n\ndraw(surface((2*I + 2*J)--(2*I - 2*J)--(-2*I - 2*J)--(-2*I + 2*J)--cycle),paleyellow,nolight);\ndraw((2*I + 2*J)--(2*I - 2*J)--(-2*I - 2*J)--(-2*I + 2*J)--cycle);\ndraw((0,0,0)--(-0.5,1.5,1));\ndraw((0,0,0)--0.8*(-0.5,1.5,1),Arrow3(6));\ndraw((0,0,0)--1.2*(-0.5,-1.5,-1),dashed);\ndraw(1.2*(-0.5,-1.5,-1)--2*(-0.5,-1.5,-1));\ndraw((0,0,0)--(-0.5,1.5,0));\n\nlabel("$\\theta$", 0.5*(-0.5,1.5,0.0) + (0,0,0.3));\n\ndot((0,0,0));\n//\n[/asy]'
)
solution_1 = remove_asy(
    'The direction vector of the line is $\\mathbf{d} = \\begin{pmatrix} 2 \\\\ 3 \\\\ 6 \\end{pmatrix},$ and the normal vector to the plane is $\\mathbf{n} = \\begin{pmatrix} -10 \\\\ -2 \\\\ 11 \\end{pmatrix}.$  Note that if $\\theta$ is the angle between $\\mathbf{d}$ in the plane, then the angle between $\\mathbf{d}$ and $\\mathbf{n}$ is $90^\\circ - \\theta.$\n\n[asy]\nimport three;\n\nsize(150);\ncurrentprojection = perspective(6,3,2);\n\ntriple I = (1,0,0), J = (0,1,0), K = (0,0,1), O = (0,0,0);\n\ndraw(surface((2*I + 2*J)--(2*I - 2*J)--(-2*I - 2*J)--(-2*I + 2*J)--cycle),paleyellow,nolight);\ndraw((2*I + 2*J)--(2*I - 2*J)--(-2*I - 2*J)--(-2*I + 2*J)--cycle);\ndraw((0,0,0)--(-0.5,1.5,1));\ndraw((0,0,0)--0.8*(-0.5,1.5,1),Arrow3(6));\ndraw((0,0,0)--1.2*(-0.5,-1.5,-1),dashed);\ndraw(1.2*(-0.5,-1.5,-1)--2*(-0.5,-1.5,-1));\ndraw((0,0,0)--(-0.5,1.5,0));\ndraw((0,0,0)--(0,0,1),Arrow3(6));\n\nlabel("$\\theta$", 0.5*(-0.5,1.5,0.0) + (0,0,0.3));\nlabel("$\\mathbf{d}$", (-0.5,1.5,1), NE);\nlabel("$\\mathbf{n}$", (0,0,1), N);\n\ndot((0,0,0));\n[/asy]\n\nTherefore,\n\\[\\cos (90^\\circ - \\theta) = \\frac{\\mathbf{d} \\cdot \\mathbf{n}}{\\|\\mathbf{d}\\| \\|\\mathbf{n}\\|} = \\frac{\\begin{pmatrix} 2 \\\\ 3 \\\\ 6 \\end{pmatrix} \\cdot \\begin{pmatrix} -10 \\\\ -2 \\\\ 11 \\end{pmatrix}}{\\left\\| \\begin{pmatrix} 2 \\\\ 3 \\\\ 6 \\end{pmatrix} \\right\\| \\left\\| \\begin{pmatrix} -10 \\\\ -2 \\\\ 11 \\end{pmatrix} \\right\\|} = \\frac{40}{7 \\cdot 15} = \\frac{8}{21}.\\]Hence, $\\sin \\theta = \\boxed{\\frac{8}{21}}.$'
)
problem_2 = remove_asy(
    "Suppose $f(x)=\\frac{3}{2-x}$. If $g(x)=\\frac{1}{f^{-1}(x)}+9$, find $g(3)$."
)
solution_2 = remove_asy(
    "Substituting $f^{-1}(x)$ into our expression for $f$, we get \\[\\frac{3}{2-f^{-1}(x)}=x.\\]Solving for $f^{-1}(x)$, we find that $f^{-1}(x)=2-\\frac{3}{x}$, so $f^{-1}(3)=2-\\frac{3}{3}=1$. Therefore, $g(3)=\\frac{1}{f^{-1}(3)}+9=\\frac{1}{1}+9=\\boxed{10}$."
)

preview(problem_1, output="png", viewer="file", filename="problem_1.png")
preview(solution_1, output="png", viewer="file", filename="solution_1.png")
preview(problem_2, output="png", viewer="file", filename="problem_2.png")
preview(solution_2, output="png", viewer="file", filename="solution_2.png")

In [28]:
joined_steps = "\n".join(steps)
print(joined_steps)

Step 1: Start with the given function f of x equals 3 over 2 minus x.
Step 2: Substitute the inverse function, f inverse of x, into the equation for f of x. So, we have 3 over 2 minus f inverse of x equals x.
Step 3: Solve for f inverse of x. To do this, multiply both sides of the equation by 2 minus f inverse of x to get rid of the denominator. This gives 3 equals x times (2 minus f inverse of x). Next, distribute x to get 3 equals 2x minus x times f inverse of x. Isolate f inverse of x by adding x times f inverse of x to both sides and dividing both sides by x. This results in f inverse of x equals 2 minus 3 over x.
Step 4: Evaluate f inverse of 3. Plug 3 into the expression we found for f inverse of x. So, f inverse of 3 equals 2 minus 3 over 3 equals 2 minus 1 equals 1.
Step 5: Find g of 3. We have g of x equals 1 over f inverse of x plus 9. Substituting 3 in for x, we get g of 3 equals 1 over f inverse of 3 plus 9. Since we found that f inverse of 3 is 1, this simplifies to g of 3

In [42]:
system = """
You are a world class math instructor.
You will be given a problem, its solution, and the detailed steps to explain the solution.
The steps are in plain english, without any math symbols.
For each step, convert it into mathematically formatted text. With proper symbols and formatting and notations.
For example, instead of writing x over y, write \\frac{x}{y}. Instead of x squared, write x^2. etc.
Use $ for math.
Return there are 5 steps. Return 5 rewritten steps.
"""

user = user_message(
    f"<problem>\n{problem_2}\n</problem>\n\n<solution>\n{solution_2}\n</solution>\n\n<steps>\n{joined_steps}\n</steps>"
)

In [43]:
math_steps = ask_oai.create(
    response_model=list[str],
    messages=[system_message(system), user],  # type: ignore
    model=ModelName.GPT_4O,
    temperature=0.4,
)

In [44]:
math_steps

['Step 1: Start with the given function $f(x) = \\frac{3}{2 - x}$.',
 'Step 2: Substitute the inverse function, $f^{-1}(x)$, into the equation for $f(x)$. So, we have $\\frac{3}{2 - f^{-1}(x)} = x$.',
 'Step 3: Solve for $f^{-1}(x)$. To do this, multiply both sides of the equation by $2 - f^{-1}(x)$ to get rid of the denominator. This gives $3 = x(2 - f^{-1}(x))$. Next, distribute $x$ to get $3 = 2x - x f^{-1}(x)$. Isolate $f^{-1}(x)$ by adding $x f^{-1}(x)$ to both sides and dividing both sides by $x$. This results in $f^{-1}(x) = 2 - \\frac{3}{x}$.',
 'Step 4: Evaluate $f^{-1}(3)$. Plug $3$ into the expression we found for $f^{-1}(x)$. So, $f^{-1}(3) = 2 - \\frac{3}{3} = 2 - 1 = 1$.',
 'Step 5: Find $g(3)$. We have $g(x) = \\frac{1}{f^{-1}(x)} + 9$. Substituting $3$ in for $x$, we get $g(3) = \\frac{1}{f^{-1}(3)} + 9$. Since we found that $f^{-1}(3)$ is $1$, this simplifies to $g(3) = \\frac{1}{1} + 9 = 1 + 9 = 10$. Therefore, the final answer is $10$.']

In [45]:
for i, step in enumerate(math_steps):
    preview(step, output="png", viewer="file", filename=f"step_{i+1}_2.png")

In [89]:
math_steps

['\\text{Step 1: Identify the direction vector of the line and the normal vector to the plane. The direction vector of the line is } \\mathbf{d} = \\begin{pmatrix} 2 \\\\ 3 \\\\ 6 \\end{pmatrix} \\text{ and the normal vector to the plane is } \\mathbf{n} = \\begin{pmatrix} -10 \\\\ -2 \\\\ 11 \\end{pmatrix} .',
 '\\text{Step 2: Understand the relationship between the angles. If } \\theta \\text{ is the angle between the direction vector and the plane, then the angle between the direction vector and the normal vector of the plane is } 90^\\circ - \\theta .',
 '\\text{Step 3: Compute the dot product of the direction vector and the normal vector. The dot product is found by multiplying corresponding components and then summing them. So, } 2 \\cdot (-10) + 3 \\cdot (-2) + 6 \\cdot 11 = -20 + (-6) + 66 = 40 .',
 '\\text{Step 4: Determine the magnitudes of the direction vector and the normal vector. The magnitude of the direction vector is } \\|\\mathbf{d}\\| = \\sqrt{2^2 + 3^2 + 6^2} = \\sq

In [84]:
new_steps = [
    "Step 1: Identify the direction vector of the line and the normal vector to the plane. The direction vector of the line is $\mathbf{d} = \begin{pmatrix} 2 \ 3 \ 6 \end{pmatrix}$ and the normal vector to the plane is $\mathbf{n} = \begin{pmatrix} -10 \ -2 \ 11 \end{pmatrix}$."
]

In [85]:
new_steps

['Step 1: Identify the direction vector of the line and the normal vector to the plane. The direction vector of the line is  \\\\mathbf{d} = \\\\begin{pmatrix} 2 \\\\\\\\ 3 \\\\\\\\ 6 \\\\end{pmatrix}  and the normal vector to the plane is  \\\\mathbf{n} = \\\\begin{pmatrix} -10 \\\\\\\\ -2 \\\\\\\\ 11 \\\\end{pmatrix} .',
 'Step 2: Understand the relationship between the angles. If  \\\\theta  is the angle between the direction vector and the plane, then the angle between the direction vector and the normal vector of the plane is  90^\\\\circ - \\\\theta .',
 'Step 3: Compute the dot product of the direction vector and the normal vector. The dot product is found by multiplying corresponding components and then summing them. So,  2 \\\\cdot (-10) + 3 \\\\cdot (-2) + 6 \\\\cdot 11 = -20 + (-6) + 66 = 40 .',
 'Step 4: Determine the magnitudes of the direction vector and the normal vector. The magnitude of the direction vector is  \\\\|\\\\mathbf{d}\\\\| = \\\\sqrt{2^2 + 3^2 + 6^2} = \\\\

In [88]:
for step in new_steps:
    try:
        preview(step, output="png")
    except Exception as e:
        print(step)
        print(e)

Step 2: Understand the relationship between the angles. If  \\theta  is the angle between the direction vector and the plane, then the angle between the direction vector and the normal vector of the plane is  90^\\circ - \\theta .
'latex' exited abnormally with the following output:
b"This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=latex)\n restricted \\write18 enabled.\nentering extended mode\n(./texput.tex\nLaTeX2e <2020-02-02> patch level 2\nL3 programming layer <2020-02-14>\n(/usr/share/texlive/texmf-dist/tex/latex/standalone/standalone.cls\nDocument Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan\ndalone\n(/usr/share/texlive/texmf-dist/tex/latex/tools/shellesc.sty)\n(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifluatex.sty\n(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty))\n(/usr/share/texlive/texmf-dist/tex/latex/xkeyval/xkeyval.sty\n(/usr/share/texlive/texmf-dist/tex/generic/xkeyval/xkeyval.tex\n(/usr/

In [6]:
creds = authenticate(token_file=TOKEN_FILE, client_secrets_file=CLIENT_SECRET_FILE)
drive_service = build(serviceName="drive", version=DRIVE_API_VERSION, credentials=creds)
slides_service = build(
    serviceName="slides", version=SLIDES_API_VERSION, credentials=creds
)

In [7]:
# drive_id = create_drive_folder(drive_service=drive_service, folder_name="VU")
drive_id = "1FVbfmu1ME29EAIKRDC-UBwFvCdM3D_WJ"

In [8]:
image_urls = upload_images_to_drive(
    drive_service=drive_service,
    file_paths=["problem_1.png", "solution_1.png", "problem_2.png", "solution_2.png"],
    folder_id=drive_id,
)

In [9]:
image_urls

['https://drive.google.com/uc?id=1G8p_gEauAtNc7-_39EfdRB8Q9c3P107u&export=download',
 'https://drive.google.com/uc?id=1s1dsahH6hPW1WtVNyN6ek_YsT86EnDLV&export=download',
 'https://drive.google.com/uc?id=1vpPufcty38QhjZ9qxcp1UIpKHTqHM8B6&export=download',
 'https://drive.google.com/uc?id=1pIu_RL2WMTGT7L6c86PCnI42Q5U7TZQk&export=download']

In [10]:
presentation_id = create_presentation(
    service=slides_service, name="VU", title="MATH 102"
)

Created presentation with ID:1NgbyXlvqsCNaIQjMpdLfd4wPfyiOga7wevjgxqngrCM


In [13]:
res = add_images(
    slides_service=slides_service,
    presentation_id=presentation_id,
    page_ids="math_slide",
    image_urls=image_urls,
    image_titles=["Problem", "Solution", "Problem", "Solution"],
)

Images and titles added successfully
